<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Analysis of NBA BasketBall Data
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>

<p style = 'font-size:16px;font-family:Arial'>
The National Basketball Association (NBA) is the premier professional basketball league in North America and one of the most popular sports organizations globally. Founded in 1946, the NBA features 30 teams (29 in the U.S. and 1 in Canada) and showcases many of the world’s top basketball athletes. Over the decades, it has evolved into a global entertainment powerhouse, with millions of fans following games, players, and stats. The comprehensive collection of game- and player-related data spanning multiple seasons are all captured and stored. It is ideal for statistical analysis, player performance evaluation, historical comparisons, and building machine learning models for sports analytics. Below business usecases can be created with the data captured, few examples are mentioned below</p>
<p style = 'font-size:18px;font-family:Arial'><b>Business usecase and value of data analysis</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Player Performance and Scouting Analytics: Identify top-performing players, rising stars, or undervalued draft prospects.</li>
    <li>Game Summary Automation and Highlights: Automatically generate narrative game summaries</li>
    <li>Advanced Fan Engagement and Fantasy Tools:  Build smart fantasy assistants or trivia bots.</li>
    <li>Operations and Game Scheduling Insights: Optimize game scheduling, referee assignments, and arena planning.</li>
    <li>Historical and Trend Analysis: Study league evolution, team rivalries, or rule impacts over time.</li>
 </ul>


<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>1.Connect to Vantage, Import python packages and explore the dataset</b></p>


In [ ]:
%%capture
# '%%capture' suppresses the display of installation steps of the following packages

!pip install -r requirements.txt --quiet

<div class="alert alert-block alert-info">
    <p style = 'font-size:16px;font-family:Arial'><b>Note: </b><i>The above statements will install the required libraries to run this demo. Be sure to restart the kernel after executing the above lines to bring the installed libraries into memory. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>
    </div>

<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial'><b>1.1 Import the required libraries</b></p>

<p style = 'font-size:16px;font-family:Arial'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
#import libraries
import matplotlib.pyplot as plt 
import getpass

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

from teradataml import *

display.max_rows=5

<p style = 'font-size:16px;font-family:Arial'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=NBA_Data_Analysis_Python.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:18px;font-family:Arial'> <b>Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. We have the option of either running the demo using foreign tables to access the data without using any storage on our environment or downloading the data to local storage, which may yield somewhat faster execution. However, we need to consider available storage. There are two statements in the following cell, and one is commented out. We may switch which mode we choose by changing the comment string.</p>

In [ ]:
%run -i ../run_procedure.py "call get_data('DEMO_BasketBall_local');"
 # takes about 2 minutes minutes, estimated space: 200Mb
#%run -i ../run_procedure.py "call get_data('DEMO_BasketBall_cloud');" 
# takes about 30 seconds, estimated space: 0Mb

<p style = 'font-size:16px;font-family:Arial'>Optional step – We should execute the below step only if we want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>2. Data Exploration on Draft Data</b></p>

<p style = 'font-size:16px;font-family:Arial'>Let us start by listing all tables/datasets available for us. </p>

In [ ]:
db_list_tables('DEMO_BasketBall')

<p style = 'font-size:16px;font-family:Arial'>We can answer most of the drafting related questions from the Draft_History table. Questions like
<ul     style = 'font-size:16px;font-family:Arial'>
    <li>How does the NBA drafting count change over time from 1949 to 2020?</li>
    <li>What are the top 10 NBA team that drafted most number of player from university?</li>
    <li>When did the teams first start to draft players from the universities?</li>
</p>

In [ ]:
tdf=DataFrame(in_schema("DEMO_BasketBall", "Draft_History"))
tdf

In [ ]:
tdf.shape

<p style = 'font-size:18px;font-family:Arial'><b>2.1 How does the NBA drafting count change over time from 1949 to 2020?</b></p>

In [ ]:
ans1=tdf.select(['season','person_id']).groupby(['season']).agg({'person_id' : ['unique']})
ans1=ans1.assign(drop_columns=True,
                   year_drafted=ans1.season,
                   total_drafted=ans1.unique_person_id)
ans1.sort('year_drafted')

In [ ]:
ans1.plot(x=ans1.year_drafted,
          y=ans1.total_drafted,
          title="Number of players drafted each year")

<p style = 'font-size:18px;font-family:Arial'><b>2.2 What are the top 10 NBA team that drafted most number of player from university?</b></p>

In [ ]:
d1=CategoricalSummary(data=tdf,target_columns ='organization_type')
d1.result

In [ ]:
from sqlalchemy import func
df2 = tdf[tdf.organization_type=='College/University']
ans2=df2.select(['team_city','team_name','person_id']).groupby(['team_city','team_name']).agg({'person_id' : ['unique']})
ans2=ans2.assign(drop_columns=True,
                   nba_team=func.concat(ans2.team_city.expression, ' ',ans2.team_name.expression),
                   total_drafted=ans2.unique_person_id)

In [ ]:
ans2.sort('total_drafted',ascending=False)

<p style = 'font-size:18px;font-family:Arial'><b>2.3 When did the teams first start to draft players from the universities?</b></p>

In [ ]:
ans3=df2.select(['team_city','team_name','season']).groupby(['team_city','team_name']).agg({'season' : ['min']})
ans3=ans3.assign(drop_columns=True,
                   nba_team=func.concat(ans3.team_city.expression, '_',ans3.team_name.expression),
                   min_year_uni_drafted=ans3.min_season)

In [ ]:
ans3.sort(['min_year_uni_drafted','nba_team'])

<p style = 'font-size:18px;font-family:Arial'><b>2.4 Where are the players coming from? </b></p>

In [ ]:
d1.result

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>3. Getting the answers using the LLM and RAG</b></p>
<p style = 'font-size:16px;font-family:Arial'>In the above section we saw that we can get the answers of the drafting questions by writing the queries around our data. We can get similar answers and much more by using generative AI and RAG.<br>Generative Question Answering (GQA) using Generative AI combines the power of Large Language Models (LLMs) like GPT-3 or GPT-4 with Retrieval-Augmented Generation (RAG) to provide accurate, context-aware responses. While LLMs are excellent at understanding and generating human-like text, they may lack access to specific or up-to-date data. RAG enhances their capabilities by retrieving relevant information from structured databases or documents and injecting that context into the LLM's prompt. The LLM then uses both the user's question and the retrieved content to generate a coherent and informed answer, making the system suitable for enterprise search, data-driven chatbots, and dynamic knowledge applications.<br>In this section we will see how we can use them with our data.<br>Let us import libraries needed for using LLM.</p>

In [ ]:
import io
import os

import numpy as np
import pandas as pd

# LLM
import sqlalchemy
from sqlalchemy import create_engine
from langchain import PromptTemplate, SQLDatabase, LLMChain
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

<p style = 'font-size:16px;font-family:Arial'>To proceed further we will need an OpenAI API key. If we don't have one yet, we can refer to the instructions provided in this guide to obtain our OpenAI API key. </p>



<a href="..//Openai_setup_api_key/Openai_setup_api_key.md" style="text-decoration:none;" target="_blank"><button style="font-size:16px;font-family:Arial;color:#fff;background-color:#00233C;border:none;border-radius:5px;cursor:pointer;height:50px;line-height:50px;display:flex;align-items:center;">OpenAI API Key Guide <span style="margin-left:10px;">&#8658;</span></button>
</a>

In [ ]:
# enter your openai api key
api_key = getpass.getpass(prompt="\n Please Enter OpenAI API key: ")

<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial'><b>3.1 Setting up LLM</b></p>

<p style = 'font-size:16px;font-family:Arial'><b>3.1.1 Connect to databases using SQL Alchemy</b></p> </p>

In [ ]:
#  Create the vantage SQLAlchemy engine
database = "DEMO_BasketBall_db"
db = SQLDatabase(
    eng,
    schema=database,
    include_tables=["Draft_History"],
)

In [ ]:
def get_db_schema():
    table_dicts = []
    database_schema_dict = {
        "database_name": database,
        "table_name": "Draft_History",
        "column_names": tdf.columns,
    }
    table_dicts.append(database_schema_dict)

    database_schema_string = "\n".join(
        [
            f"Database: {table['database_name']}\nTable: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
            for table in table_dicts
        ]
    )

    return database_schema_string

In [ ]:
database_schema = get_db_schema()
print(database_schema)

<p style = 'font-size:16px;font-family:Arial'><b> 3.1.2 Format the answer and Display</b></p>

<p style = 'font-size:16px;font-family:Arial'>To view the answer in proper format with markdown</p>

In [ ]:
from IPython.display import display, Markdown

def response_template(response):
    if "output" in response:
        return f"<p style = 'font-size:16px;font-family:Arial'>Answer:  <b>{response['output']}<b>"
    else:
        return f"<p style = 'font-size:16px;font-family:Arial'>Answer:  <b>{response}<b>"

def error_template():
    return f"<p style = 'font-size:16px;font-family:Arial'>Sorry, there was an error while generating the SQL query. The GenAI may have made a mistake in the syntax of the query.  <br>"

<p style = 'font-size:16px;font-family:Arial'><b> 3.1.3 Define LLM model</b></p>

<p style = 'font-size:16px;font-family:Arial'></p>

In [ ]:
# OpenAI API
os.environ["OPENAI_API_KEY"] = api_key

# set LLM model
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)

<p style = 'font-size:18px;font-family:Arial'><b>3.1.4 Setup SQLAgent</b></p> 

In [ ]:
# main prompt
generated_prompt = f"""You are a Teradata Database expert and you are tasked with generating SQL queries for Teradata based on user questions. 
    Your response should ONLY be based on the given context and follow the response guidelines and format instructions.

        Utilize the following tables and columns exclusively when creating SQL queries:\n{database_schema}

        Here are some tips for writing Teradata style queries: 
        * Always use table aliases when your SQL statement involves more than one source
        * Aggregated fields like COUNT(*) must be appropriately named
        * Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 3 results by using SELECT TOP 3, note that LIMIT function does not works in Teradata DB.
        * Remove unnecessary ORDER BY clauses unless required. 
        * Remember: Do not use 'LIMIT' or 'FETCH' keyword in the SQLQuery, instead of TOP keyword, For Example: To select top 3 results, use TOP keyword instead of LIMIT or FETCH. 
        * Important: If you received the error "Bad character in format or data", change the value of columns, get the values from table only.
        * Remember: purchased column have only 2 values: 'yes' or 'no'
        * Critical Instruction: Use default database as 'DEMO_User'
        
        Few examples of SQL:
        Example1: SELECT count(*) as total_count FROM DEMO_BasketBall.Draft_History
        Example2: SELECT season as year_drafted, COUNT(DISTINCT person_id) AS total_drafted
                  FROM DEMO_BasketBall.Draft_History
                  GROUP BY year_drafted 

        Response Guidelines: 
        * Whenever possible, give the answer in bulleted points and proper markup.
        * Critical Instruction: Ensure responses are exclusively derived from query results. Refrain from generating or adding synthetic data in any form.
        * Most important: Always create a syntactically correct Teradata-style query that addresses the question, 
        even if it has been asked and answered previously. Ensure the query is generated from scratch and does not rely on any pre-existing data stored in memory.
      

        Given a user's question about this data, write a valid Teradata SQL query that accurately extracts or calculates the requested information from these tables and adheres to SQL best practices for Teradata database, optimizing for readability and performance where applicable.
        Most important: Execute the SQL and return the final answer only in simple english statement. 
        Critical Instruction: Do not return json or SQL."""


messages = [
    HumanMessagePromptTemplate.from_template("{input}"),
    AIMessage(content=generated_prompt),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
]

prompt = ChatPromptTemplate.from_messages(messages)


agent_executor = create_sql_agent(
    llm,
    db=db,
    agent_type="openai-tools",
    verbose=False,
    prompt=prompt,
    max_iterations=10,
    max_execution_time=20,
    handle_parsing_errors=True,
    return_intermediate_steps=True,
    handle_sql_errors=True,
    max_tokens=4000,
)

In [ ]:

try:
    # Enter the query
    query = """Where are the players coming from?"""

    # Response from Langchain
    response = agent_executor.invoke(query)

    display(Markdown(response_template(response)))
except:
    display(Markdown(error_template()))

<p style = 'font-size:16px;font-family:Arial'>From above answer we can see that the LLM generates a response taking the menaning of the questions as it understands them. For getting the similar query and answer as in section 2.1 we should modify our question to as below.</p>

In [ ]:
try:
    # Enter the query
    query = """Give me the count of players coming from each organization type?"""

    # Response from Langchain
    response = agent_executor.invoke(query)

    display(Markdown(response_template(response)))
except:
    display(Markdown(error_template()))

<p style = 'font-size:16px;font-family:Arial'>From above we can see that we can get similar answers from the LLM quering the database as we get when we query the database ourself.<br>The true power of Large Language Models (LLMs) lies not merely in retrieving data, but in their ability to infer meaning, context, and narrative from structured information and present it in fluent, natural language. Unlike traditional SQL queries that return raw data points, LLMs can synthesize this information, recognize patterns, and articulate insights in a way that aligns with human understanding. This capability is particularly valuable when working with complex, event-driven datasets such as the Play_By_Play table in the NBA dataset. This table captures granular, timestamped actions during a basketball game—such as shots, fouls, substitutions, and timeouts—across multiple players and teams. On its own, the raw data can be overwhelming and lacks cohesion. However, when this data is processed using a Retrieval-Augmented Generation (RAG) framework, the LLM can transform it into a meaningful, human-readable game summary or analysis—demonstrating how generative AI goes beyond querying to storytelling and insight generation.</p>

In [ ]:
play_data = DataFrame(in_schema("DEMO_BasketBall", "Play_By_Play"))
play_data

<p style = 'font-size:16px;font-family:Arial'>Let us take data for a game </p>

In [ ]:
play_data[play_data.game_id == '42200174'].sort('eventnum')


<p style = 'font-size:16px;font-family:Arial'>Here we can see the data returned by the query but we cannot make much sense to it or get a summary on what happened in this game. We can get summaries of the game by using the LLMs. </p>

In [ ]:
# Define Helper: Mapping for Event Types
eventmsgtype_map = {
    1: "Field Goal Made",
    2: "Field Goal Missed",
    3: "Free Throw Attempt",
    4: "Rebound",
    5: "Turnover",
    6: "Foul",
    7: "Violation",
    8: "Substitution",
    9: "Timeout",
    10: "Jump Ball",
    11: "Ejection",
    12: "Start of Period",
    13: "End of Period",
    18: "Unknown Event"
}

eventmsgactiontype_map = {
    1: "Jumpshot",
    2: "Lost Ball Turnover",
    4: "Traveling / Offensive Foul",
    5: "Layup",
    7: "Dunk",
    10: "Free Throw 1-1",
    11: "Free Throw 1-2",
    12: "Free Throw 2-2",
    40: "Out of Bounds",
    41: "Block/Steal",
    42: "Driving Layup",
    50: "Running Dunk",
    52: "Alley Oop Dunk",
    55: "Hook Shot",
    57: "Driving Hook Shot",
    58: "Turnaround Hook Shot",
    66: "Jump Bank Shot",
    71: "Finger Roll Layup",
    72: "Putback Layup",
    108: "Cutting Dunk Shot"
}

# Define Function to Summarize a Game
def summarize_game(game_id):
    query = f"""
    SELECT homedescription, neutraldescription, visitordescription, eventmsgtype, eventmsgactiontype
    FROM DEMO_BasketBall.Play_by_Play
    WHERE game_id = '{game_id}'
    ORDER BY eventnum
    """
    df = pd.read_sql(query, eng)

    # Translate event types
    df['event_type'] = df['eventmsgtype'].map(eventmsgtype_map)
    df['action_type'] = df['eventmsgactiontype'].map(eventmsgactiontype_map)

    # Build a raw text summary
    summary = []
    for _, row in df.iterrows():
        desc = row['homedescription'] or row['neutraldescription'] or row['visitordescription']
        if desc:
            text = f"[{row['event_type']} / {row['action_type']}] {desc}"
            summary.append(text)

    # Combine into one large context
    full_summary = "\n".join(summary)
    return full_summary

# Define Function to generate Natural Language Summary
def generate_nba_game_summary(game_id):
    context = summarize_game(game_id)
    prompt = f"""
    Summarize the following NBA game events in a detailed but readable style for a fan:

    {context}

    Write it as a smooth flowing game recap.
    """
    return llm.predict(prompt)
    #return llm.invoke(prompt)

<p style = 'font-size:16px;font-family:Arial'>Now let us get the summary for the same game from LLM.</p>

In [ ]:
example_game_id = '42200174'
summary_text = generate_nba_game_summary(example_game_id)

In [ ]:
display(Markdown(response_template(summary_text)))

<p style = 'font-size:20px;font-family:Arial'><b>Conclusion</b></p>
<p style = 'font-size:16px;font-family:Arial'>In this demo we have seen how we can agument RAG and LLMs with our anaylsis of datasets to give us insights.</p>

<hr style='height:2px;border:none;'>
<b style = 'font-size:28px;font-family:Arial'>4. Cleanup</b>

<hr style='height:1px;border:none;'>
<p style = 'font-size:18px;font-family:Arial'> <b>4.1 Databases and Tables </b></p>

<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_BasketBall');"        # Takes 5 seconds

In [ ]:
remove_context()

<b style = 'font-size:28px;font-family:Arial'>Dataset:</b>
NBA Dataset
- `30` teams
- `4800+` players
- `65,000+` games (every game since the inaugural 1946-47 NBA season)
- `Box Scores` for over 95% of all games
- `Play-by-Play` game data from 01-Jan-2020 till 30-Jun-2023 of Play-by-Play data in all!

<p style = 'font-size:16px;font-family:Arial'><b>Dataset source:</b> <a href = 'https://www.kaggle.com/datasets/wyattowalsh/basketball'>kaggle</a></p>

<p style = 'font-size:16px;font-family:Arial'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/r/Enterprise_IntelliFlex_VMware/Teradata-Package-for-Python-User-Guide-17.20'>here</a></li>
    <li>Langchain Python reference: <a href='https://python.langchain.com/docs/get_started/introduction/'>here</a></li>
</ul>

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>